
# Example of creation of a configuration and preparation of simulations

In this example we will see how to import a configuration from a structure file and modify it.

The steps are:

1) Create a new project object;  
2) Add a system object to the project;  
3) Import a configuration;
4) Modify the system;

Details on how to create the files needed to run simulations are shown in the Notebook **"Study_torsional_angles.ipynb"**


## 0. Load the necessary modules:

In [1]:
from sim_launch_py.classes import Project
from sim_launch_py.utilities import get_distance
import numpy as np
import nglview as nv
import os

def makeview(models, showBox=True):
    import math
    views=[]
    for model in models:
        view=nv.show_structure_file(model)
        view._remote_call("setSize",target="Widget",
                         args=["{}px".format(500/len(models)),"{}px".format(500/len(models))])
        view.parameters=dict(backgroundColor='white',clipDist=-100,
                            sampleLevel=3)
        if showBox:
            view.add_unitcell()
        view.camera='orthographic'
        views.append(view)
    import ipywidgets
    hboxes = [ipywidgets.HBox(views[i*2:i*2+2])
             for i in range(int(math.ceil(len(views)/2.0)))]
    vbox=ipywidgets.VBox(hboxes)
    return vbox

## 1. Create a new project

Here we create a new project with the name **'import_configuration'**. This creates a new directory at the indicated path. Please note that the overwrite=True flag deletes an already existent directory at the given path.

When a project is created, the program will look for Gromacs and AmberTools binaries in your PATH. If they are not found, a prompt will ask to manually indicate them.

In [2]:
pname='import_configuration'
ppath='./{}'.format(pname)

p=Project.new_project(pname,ppath,overwrite=True)

New Project: import_configuration


## 2. Add a system

Here we create a system named **'seed_form_IV'** in the project using the *Project.add_system()* method.  This command creates a directory with the same name of the system inside the *project/path/Systems* directory.

In [3]:
# Create a system

p.add_system('seed_form_IV')
s=p.systems[0]

**************************************************
Created system seed_form_IV
**************************************************


## 3. Import configuration

While in the **'Study_torsional_angles.ipynb'** example we have built a system from scratch, here we will create the initial configurations importing the coordinates from a structure file.

In [4]:

s.add_molecule('olanzapine', structure_file='/home/matteo/Work/organic_molecules_simulations/examples/Structures/form_IV.pdb')

box=makeview(['/home/matteo/Work/organic_molecules_simulations/examples/Structures/form_IV.pdb'],
             showBox=True)
box

## 4. Modify configuration

### 4.1 Replicate cell

The aim of this tutorial is to generate a seed made by the crystal molecules. To do so the first step after importing the unit cell is to replicate it in space to generate a bulk.

In [5]:
s.replicate_cell(repl=[4,3,3])

s.save_pdb('replicated_cell.pdb')

box=makeview([s.last_saved_structure],showBox=True)
box

### 4.2 Remove excess molecules

Then we can eliminate the excess molecules to keep the seed of the desired shape and size. In this example we keep a spherical seed composed by all molecules with the center of mass within 1.5 nm of the center of the box.

In [6]:
cob=np.array([0. for i in range(3)])

n=0
for m in s.molecules:
    for a in m.atoms:
            cob+=a.coordinates
            n+=1

cob/=n

delete_mols=[]
cutoff=1.5 #nm
for m in s.molecules:
    if get_distance(m.com,ref=cob,box=s.box)>cutoff:
        delete_mols.append(m.index)

s.delete_molecule(delete_mols)
s.save_pdb('remaining_mols.pdb')

box=makeview([s.last_saved_structure])
box

Deleting 108 molecules from system seed_form_IV.
Now system seed_form_IV has 36 molecules


### 4.3 Center molecules in the box

We can now change the shape and size of the box and recenter the seed.  
We will use a cubic box of 6
nm of side

In [7]:
s.add_box(6,shape='cubic')
s.center_box()
s.save_pdb('centered.pdb')

box=makeview([s.last_saved_structure])
box

### 4.4 Insert solvent molecules

We will now insert solvent molecules, water and ethanol at a given composition.

In [8]:
seed_com=np.array([0., 0., 0.])

n=0
for m in s.molecules:
    for a in m.atoms:
        seed_com+=a.coordinates
        n+=1
    
seed_com/=n

s.create_group('seed',molecules=s.molecules)

# estimate in some way the number of solvent molecules to add

nwat_to_add=1100
netoh_to_add=1250

# add solvent molecules

s.insert_molecules('water','/home/matteo/Work/organic_molecules_simulations/examples/Structures/water/water.pdb',
                   initial_conf='centered.pdb',final_conf='with_water.pdb',nmol=nwat_to_add)

# once water molecules have been inserted, remove those that could be within the seed

water_mols=s.find_molecule_by_resname('WAT')

delete_mols=[]
for wm in s.molecules[water_mols[0]:water_mols[-1]]:
    if get_distance(wm.com,ref=seed_com, box=s.box)<=cutoff:
        delete_mols.append(wm.index)

s.delete_molecule(delete_mols)

# rinse and repeat if we need a precise number of solvent molecules. something like "while [nmols solvent < target nmols]; insert molecules; remove those that are in forbidden positions"

s.save_pdb('removed_water_in_seed.pdb')


s.insert_molecules('ethanol','/home/matteo/Work/organic_molecules_simulations/examples/Structures/ethanol/ethanol.pdb',initial_conf='removed_water_in_seed.pdb',final_conf='with_ethanol.pdb',nmol=netoh_to_add)

etoh_mols=s.find_molecule_by_resname('ETH')

delete_mols=[]
for etohm in s.molecules[etoh_mols[0]:etoh_mols[-1]]:
    if get_distance(etohm.com,ref=seed_com,box=s.box)<=cutoff:
        delete_mols.append(etohm.index)

s.delete_molecule(delete_mols)


1100 molecules of residue WAT have been added. The file Systems/seed_form_IV/with_water.pdb has been created.
Deleting 10 molecules from system seed_form_IV.
Now system seed_form_IV has 1126 molecules
1250 molecules of residue ETH have been added. The file Systems/seed_form_IV/with_ethanol.pdb has been created.
Deleting 11 molecules from system seed_form_IV.
Now system seed_form_IV has 2365 molecules


In [9]:

box=makeview(['Systems/seed_form_IV/with_water.pdb',
              'Systems/seed_form_IV/with_ethanol.pdb'])
box

## Save the project

In [10]:
p.save()


Saving Project...done
